In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
!pip install transformers
import nltk
nltk.download('punkt')
# !pip install nltk


In [ ]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import random
from nltk import sent_tokenize
from transformers import BertForSequenceClassification

df = pd.read_csv('Issue_tree_sentiment__no_error (1).tsv', delimiter = '\t', quoting = 3)
# df.head()
# data=pd.read_excel('issue.xlsx')

In [ ]:
df.head()

In [ ]:
data=df[['Issue Experienced', 'Sub Reason']]
data['Sub Reason'].value_counts()

SparksandPromotions                4922
Other                              3878
Navigation                         3851
Stock                              3555
CheckoutandLogin                   2828
ProductRange                       2521
DeliveryInformationandMyAccount    2110
MissingContentorImagery            1614
OrderandCommunication              1544
Search                             1453
TechnicalIssue                      821
SitePerformance                     536
General                             136
COVID-19                             33
Covid                                 5
other                                 5
navigation                            5
checkoutandlogin                      4
search                                4
sparksandpromotions                   3
checkoutandLogin                      3
productrange                          2
stock                                 2
missingcontentorimagery               2
deliveryinformationandmyaccount       1


In [ ]:
data['Sub Reason']=data['Sub Reason'].apply(lambda x: str(x).lower())
data.loc[data['Sub Reason'] == 'covid', 'Sub Reason'] = 'covid-19'
possible_labels = data['Sub Reason'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)
data['Sub Reason'].value_counts()


{'siteperformance': 0, 'stock': 1, 'orderandcommunication': 2, 'search': 3, 'navigation': 4, 'other': 5, 'deliveryinformationandmyaccount': 6, 'sparksandpromotions': 7, 'productrange': 8, 'missingcontentorimagery': 9, 'technicalissue': 10, 'checkoutandlogin': 11, 'covid-19': 12, 'general ': 13}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

sparksandpromotions                4925
other                              3883
navigation                         3856
stock                              3557
checkoutandlogin                   2835
productrange                       2523
deliveryinformationandmyaccount    2111
missingcontentorimagery            1616
orderandcommunication              1544
search                             1457
technicalissue                      821
siteperformance                     536
general                             136
covid-19                             39
Name: Sub Reason, dtype: int64

In [ ]:
data['label'] = data['Sub Reason'].replace(label_dict)
print(data['Sub Reason'].value_counts())
ext=[]
for i,j in  data['Sub Reason'].value_counts().items():
    if j <1000:
        # ext.append(i)
        data.loc[data['Sub Reason'] == i, 'Sub Reason'] = 'other'
print(data['Sub Reason'].value_counts())

sparksandpromotions                4925
other                              3883
navigation                         3856
stock                              3557
checkoutandlogin                   2835
productrange                       2523
deliveryinformationandmyaccount    2111
missingcontentorimagery            1616
orderandcommunication              1544
search                             1457
technicalissue                      821
siteperformance                     536
general                             136
covid-19                             39
Name: Sub Reason, dtype: int64
other                              5415
sparksandpromotions                4925
navigation                         3856
stock                              3557
checkoutandlogin                   2835
productrange                       2523
deliveryinformationandmyaccount    2111
missingcontentorimagery            1616
orderandcommunication              1544
search                             1457
Name: Sub

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
new_aug_data=[]
for i in data[data['label'].isin(ext)].itertuples():
    
    aug_ext=set(eda(i._1,alpha_rs=0.00001,p_rd=0.00,alpha_ri=0.001))
    
    for j in aug_ext:
        dum=[]
        dum.append(j)
        dum.append(i._2)
        dum.append(i.label)
        new_aug_data.append(dum)
print(data['Sub Reason'].value_counts())

In [ ]:
new_df=pd.DataFrame(new_aug_data,columns=['Issue Experienced','Sub Reason','label'])
print(new_df['Sub Reason'].value_counts())
data=pd.concat([data,new_df])
print(data['Sub Reason'].value_counts())

In [ ]:
data.loc[data['label'].isin([5,10,12,13]), 'label'] = 0

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
data=data[data['Sub Reason']!='other']

In [ ]:
data.head(1)

,Issue Experienced,Sub Reason,label
1,After ordering yesterday a skirt and jumper I ...,stock,1


In [ ]:
shuf_data=[]
for i,row in data.iterrows():
  dum=[]
  sent=sent_tokenize(row['Issue Experienced'])
  if len(sent)>1:
    random.shuffle(sent)
    dum.append(''.join(sent))
    dum.append(row['Sub Reason'])
    dum.append(row['label'])
    shuf_data.append(dum)

In [ ]:
sf=pd.DataFrame(shuf_data,columns=data.columns)

In [ ]:
data=pd.concat([data,sf])

In [ ]:
print(data['Sub Reason'].value_counts())

sparksandpromotions                7812
other                              7612
navigation                         6155
stock                              5977
checkoutandlogin                   4623
productrange                       4338
deliveryinformationandmyaccount    3594
missingcontentorimagery            2715
orderandcommunication              2548
search                             2337
Name: Sub Reason, dtype: int64


In [ ]:
data.shape

(40099, 3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

data['data_type'] = ['not_set']*data.shape[0]

data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'


In [ ]:
data.groupby(['Sub Reason','label', 'data_type']).count()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train']['Issue Experienced'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256,
    truncation=True, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='val']['Issue Experienced'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    truncation=True, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
                                          
# encoded_data_train = tokenizer.batch_encode_plus(
#     data[data.data_type=='train']['filtered SR'].values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=500,
#     truncation=True, 
#     return_tensors='pt'
# )

# encoded_data_val = tokenizer.batch_encode_plus(
#     data[data.data_type=='val']['filtered SR'].values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=500, 
#     truncation=True, 
#     return_tensors='pt'
# )


# input_ids_train = encoded_data_train['input_ids']
# attention_masks_train = encoded_data_train['attention_mask']
# labels_train = torch.tensor(data[data.data_type=='train'].label.values)

# input_ids_val = encoded_data_val['input_ids']
# attention_masks_val = encoded_data_val['attention_mask']
# labels_val = torch.tensor(data[data.data_type=='val'].label.values)

# dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
# dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# set(eda('i didnot receive the gift', alpha_rs=0.00001,p_rd=0.00,alpha_ri=0.001))
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
data['Sub Reason'].value_counts()

sparksandpromotions                7812
navigation                         6155
stock                              5977
checkoutandlogin                   4623
productrange                       4338
deliveryinformationandmyaccount    3594
missingcontentorimagery            2715
orderandcommunication              2548
search                             2337
Name: Sub Reason, dtype: int64

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train,
                        desc="Epoch {:1d}".format(epoch),
                        leave=False,
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids'       : batch[0],
            'attention_mask'  : batch[1],
            'labels'          : batch[2]
        }
    
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix( {'training_loss': '{:3f}'.format(loss.item() / len(batch))} )

    torch.save(model.state_dict(), 'BERT_ft_epoch{}.model'.format(epoch))
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write('Training loss: {}'.format(loss_train_avg))
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    val_acc=accuracy_per_class(predictions, true_vals)
    
    tqdm.write('Validation loss: {}'.format(val_loss))
    tqdm.write('f1 score (weighted): {}'.format(val_f1))
    # tqdm.write(val_acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.



Epoch {epoch}
Training loss: 0.9376589711014938



Class: stock
Accuracy: 1221/1424

Class: orderandcommunication
Accuracy: 420/579

Class: search
Accuracy: 344/566

Class: navigation
Accuracy: 1177/1471

Class: deliveryinformationandmyaccount
Accuracy: 542/815

Class: sparksandpromotions
Accuracy: 1611/1819

Class: productrange
Accuracy: 771/1012

Class: missingcontentorimagery
Accuracy: 531/632

Class: checkoutandlogin
Accuracy: 845/1043

Validation loss: 0.7737201620149194
f1 score (weighted): 0.7965717678792011


In [ ]:

# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=len(label_dict),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

In [ ]:
import random
from nltk import sent_tokenize
sen=sent_tokenize('I am b2. hello i am going to kill you. a ')
random.shuffle(sen)
sen

['I am b2.', 'a', 'hello i am going to kill you.']

In [ ]:
random.shuffle(['a','b','c'])

TypeError: ignored